# Sentiment140 preprocessing and feature extraction

In [1]:
import dask
import dask.dataframe
import dask.diagnostics
import dask.multiprocessing
import langid
import os
import pandas as pd
from urllib.request import urlretrieve
import spacy
import ssl
import sys
from tqdm import tqdm_notebook as tqdm
from zipfile import ZipFile

LOCAL_PATH = '../data/sentiment140'
PATH_TO_ARCHIVE = os.path.join(LOCAL_PATH, 'archive.zip')

os.makedirs(LOCAL_PATH, exist_ok=True)

## Download archive

In [2]:
with tqdm() as progress:
    def report(count, blockSize, totalSize):
        progress.total = totalSize
        progress.update(blockSize)
    ssl._create_default_https_context = ssl._create_unverified_context
    urlretrieve('http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip',
                PATH_TO_ARCHIVE, reporthook=report)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

## Extract and tidy data

In [3]:
with ZipFile(PATH_TO_ARCHIVE) as zip_file:
    df = pd.read_csv(zip_file.open('training.1600000.processed.noemoticon.csv'),
                    encoding='latin1', names=['target', 'id', 'date', 'flag', 'user', 'tweet'])

df['sentiment'] = df['target'].map({0: 'negative',
                                    2: 'neutral',
                                    4: 'positive'})

df = df[['sentiment', 'user', 'tweet']]

## Filter non-English tweets

In [4]:
dd = dask.dataframe.from_pandas(df, npartitions=100)
with dask.diagnostics.ProgressBar():
    df['langid'] = dd.map_partitions(
        lambda dataframe: dataframe.apply(
            (lambda row: langid.classify(row['tweet'])[0]), axis=1)) \
        .compute(get=dask.multiprocessing.get)

df = df[df['langid'] == 'en'].reindex()

[########################################] | 100% Completed | 13min 48.5s


## Process with spaCy

In [ ]:
nlp = spacy.load('en_core_web_lg')
df['doc'] = list(tqdm(nlp.pipe(df['tweet'], n_threads=-1), total=len(df)))

HBox(children=(IntProgress(value=0, max=1521182), HTML(value='')))